In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:0"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(2, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        mask = (torch.triu(torch.ones(sz, sz, device = device)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        #mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment:    
    def reset(self, npoints, batchsize, nsamples=1):
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints, 2], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints, 2)
        )
        
        self.distance_matrix = (self.points.unsqueeze(1) - self.points.unsqueeze(2)).square().sum(-1).sqrt() # [batchsize * nsamples, npoints, npoints]
        
        self.previous_point = None
        
        self.points_mask = torch.zeros(
                    [self.batchsize, npoints], dtype=torch.bool, device=device
                )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert list(point_index.size()) == [self.batchsize]
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        if self.previous_point != None:
            self.cost += self.distance_matrix.gather(2, self.previous_point.unsqueeze(1).unsqueeze(2).expand(-1, self.npoints, 1)).squeeze(2).gather(1, point_index.unsqueeze(1)).squeeze(1)
        
        self.previous_point = point_index
        self.points_mask.scatter_(1, point_index.unsqueeze(1), True)
        self.points_sequence = torch.cat([self.points_sequence, point_index.unsqueeze(1)], dim = 1)
        
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    
    def laststep(self):
        
        assert self.points_sequence.size(1) == self.npoints
        
        self.cost += self.distance_matrix.gather(2, self.points_sequence[:, 0].unsqueeze(1).unsqueeze(2).expand(-1, self.npoints, 1)).squeeze(2).gather(1, self.points_sequence[:, -1].unsqueeze(1)).squeeze(1)
    

env = environment()


def train(epochs = 30000, npoints = 10, batchsize = 100, nsamples = 8, negative_cutoff = 1):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(0, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        env.laststep()
        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > negative_cutoff
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        #loss = ( 
        #        - logprob.view(batchsize, nsamples)[:, :-1] 
        #        #* (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
        #        * positive_reinforcement_binary 
        #        + logprob.view(batchsize, nsamples)[:, :-1] 
        #        * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 80) 
        #        * negative_reinforcement_binary 
        #).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
   

In [ ]:
train(300000, 100, 256, 8, 1)

10.165550231933594 -34.38252639770508 -63.00194549560547 -33.33745193481445 -67.34614562988281 -67.34613037109375
10.143495559692383 -34.37852096557617 -63.72127151489258 -30.45086669921875 -84.56082153320312 -84.56082153320312
10.119187355041504 -34.503990173339844 -63.47574996948242 -34.53193664550781 -77.99797058105469 -77.99794006347656
10.086257934570312 -34.38628005981445 -63.39647674560547 -33.9175910949707 -55.97492599487305 -55.974937438964844
10.114106178283691 -34.34992218017578 -63.212196350097656 -39.015438079833984 -60.320526123046875 -60.32050323486328
10.102103233337402 -34.39280700683594 -63.19534683227539 -32.365779876708984 -48.602874755859375 -48.60289001464844
10.080995559692383 -34.1531867980957 -63.16118621826172 -36.24456024169922 -57.381591796875 -57.3816032409668
10.11338996887207 -34.37371063232422 -63.083717346191406 -39.30935287475586 -65.20604705810547 -65.20609283447266
10.037307739257812 -34.132205963134766 -62.346099853515625 -31.543031692504883 -65.941

9.530017852783203 -26.138002395629883 -49.43717575073242 -26.1671085357666 -45.92564392089844 -45.9256477355957
9.525367736816406 -26.472232818603516 -49.570926666259766 -28.195575714111328 -48.16675567626953 -48.166709899902344
9.537120819091797 -26.330947875976562 -49.824989318847656 -25.909133911132812 -54.09226989746094 -54.092254638671875
9.5225830078125 -26.581605911254883 -49.77275085449219 -28.43603515625 -39.213653564453125 -39.2136116027832
9.535486221313477 -26.48624038696289 -50.12137985229492 -28.93512535095215 -52.71428298950195 -52.7142333984375
9.524383544921875 -26.496591567993164 -49.918678283691406 -22.81976318359375 -56.64332580566406 -56.64328384399414
9.510662078857422 -26.29198455810547 -50.13518524169922 -27.149394989013672 -53.14492416381836 -53.144927978515625
9.519381523132324 -25.84857177734375 -49.10354232788086 -24.48379135131836 -42.973785400390625 -42.9737548828125
9.573352813720703 -26.065162658691406 -49.3580436706543 -29.149131774902344 -50.5302162170

9.393157958984375 -21.18441390991211 -40.65656661987305 -21.067180633544922 -36.4815673828125 -36.481571197509766
9.353479385375977 -20.88462257385254 -40.28435516357422 -19.50341796875 -38.282283782958984 -38.282310485839844
9.394033432006836 -20.829551696777344 -40.06358337402344 -18.17934799194336 -35.965065002441406 -35.96507263183594
9.399152755737305 -20.956157684326172 -40.33308029174805 -18.657276153564453 -30.002315521240234 -30.002225875854492
9.36892318725586 -20.592044830322266 -39.56435012817383 -20.398954391479492 -32.89179992675781 -32.89183044433594
9.401453018188477 -20.84659194946289 -39.8394775390625 -18.826860427856445 -55.954254150390625 -55.95423889160156
9.391510009765625 -20.359683990478516 -39.651100158691406 -16.356060028076172 -51.73545455932617 -51.73554611206055
9.393960952758789 -20.891502380371094 -40.25765609741211 -23.183822631835938 -44.130401611328125 -44.130435943603516
9.356487274169922 -20.938297271728516 -40.48256301879883 -22.150718688964844 -38.

9.344154357910156 -18.055347442626953 -34.840354919433594 -18.645444869995117 -41.13861083984375 -41.13858413696289
9.320491790771484 -18.268274307250977 -35.27589416503906 -18.186145782470703 -32.20648956298828 -32.20649719238281
9.386957168579102 -18.129024505615234 -35.0791130065918 -18.559539794921875 -25.763614654541016 -25.7635498046875
9.335264205932617 -18.015777587890625 -34.85911178588867 -16.265174865722656 -34.12810516357422 -34.12809753417969
9.289787292480469 -17.998992919921875 -34.9609375 -19.152801513671875 -25.352054595947266 -25.352052688598633
9.352834701538086 -17.854686737060547 -34.67055130004883 -19.469682693481445 -33.5343132019043 -33.53429412841797
9.350523948669434 -17.902801513671875 -34.42513656616211 -15.544598579406738 -35.257572174072266 -35.2575569152832
9.365785598754883 -17.79690170288086 -34.78205490112305 -14.740041732788086 -32.61028289794922 -32.61026382446289
9.346988677978516 -17.90941047668457 -34.416378021240234 -16.444156646728516 -30.065629

9.300731658935547 -16.07327651977539 -31.227306365966797 -15.044398307800293 -33.312618255615234 -33.312583923339844
9.350926399230957 -16.10405158996582 -31.442201614379883 -18.04494857788086 -25.362403869628906 -25.36237907409668
9.324786186218262 -16.430591583251953 -32.12488555908203 -23.332340240478516 -36.0697135925293 -36.06971740722656
9.35739517211914 -16.552371978759766 -32.046051025390625 -14.108423233032227 -30.25636863708496 -30.256338119506836
9.357351303100586 -16.580965042114258 -32.42509078979492 -20.707149505615234 -34.08358383178711 -34.083648681640625
9.34344482421875 -16.770095825195312 -32.66156768798828 -15.508613586425781 -40.30683898925781 -40.306888580322266
9.329693794250488 -16.78140640258789 -32.631980895996094 -17.034772872924805 -23.92293930053711 -23.92289161682129
9.35634994506836 -16.534894943237305 -32.236202239990234 -14.760189056396484 -25.686878204345703 -25.68679428100586
9.310871124267578 -16.175453186035156 -31.997665405273438 -14.92617893218994

...

8.58523178100586 -1.9274139404296875 -3.729979991912842 -1.031005859375 -2.483154296875 -2.483154296875
8.55139446258545 -1.9789209365844727 -3.814457416534424 -2.572509765625 -6.218994140625 -6.218994140625
8.605852127075195 -1.8185091018676758 -3.5156731605529785 -0.997802734375 -9.276123046875 -9.276611328125
8.575725555419922 -1.8466463088989258 -3.676337242126465 -1.744873046875 -6.56817626953125 -6.56768798828125
8.594381332397461 -1.8440254926681519 -3.7099599838256836 -2.642578125 -2.22393798828125 -2.22418212890625
8.564364433288574 -1.7837791442871094 -3.6063790321350098 -1.85888671875 -2.022705078125 -2.02325439453125
8.53842830657959 -1.886989712715149 -3.7445929050445557 -0.87451171875 -0.86572265625 -0.865478515625
8.58653736114502 -1.9491901397705078 -3.8023359775543213 -0.89208984375 -5.107177734375 -5.10888671875
8.554292678833008 -1.9165165424346924 -3.7652745246887207 -1.58837890625 -7.54931640625 -7.550048828125
8.5523681640625 -1.8574573993682861 -3.691802501678467

8.6309232711792 -2.393845558166504 -4.746535778045654 -2.46044921875 -3.061279296875 -3.0625
8.58674430847168 -2.370922327041626 -4.6121320724487305 -3.7197265625 -9.374755859375 -9.375244140625
8.640661239624023 -2.4205405712127686 -4.832393169403076 -3.559572219848633 -5.088623046875 -5.089599609375
8.62574577331543 -2.3808908462524414 -4.737236976623535 -5.081787109375 -7.877685546875 -7.876708984375
8.603723526000977 -2.349421501159668 -4.7321343421936035 -3.3369140625 -4.30615234375 -4.3056640625
8.594857215881348 -2.4186415672302246 -4.7883782386779785 -3.0347900390625 -4.75244140625 -4.7529296875
8.591506004333496 -2.463015556335449 -4.729348659515381 -2.024169921875 -3.462646484375 -3.463134765625
8.565692901611328 -2.3490641117095947 -4.487908363342285 -1.93115234375 -3.3447265625 -3.34423828125
8.618525505065918 -2.286907911300659 -4.4308671951293945 -1.264404296875 -0.9873046875 -0.98779296875
8.625585556030273 -2.2027344703674316 -4.41710901260376 -3.946533203125 -2.8085937

8.548674583435059 -2.4355788230895996 -4.7147088050842285 -4.755615234375 -3.822021484375 -3.82177734375
8.57649040222168 -2.364082098007202 -4.7031707763671875 -1.9677734375 -2.93115234375 -2.9306640625
8.561098098754883 -2.449392318725586 -4.836015701293945 -1.782470703125 -7.99169921875 -7.991455078125
8.604623794555664 -2.4143636226654053 -4.705996990203857 -3.250732421875 -2.03857421875 -2.03759765625
8.614632606506348 -2.4335267543792725 -4.8401360511779785 -2.5009765625 -3.177001953125 -3.177490234375
8.652729034423828 -2.4284610748291016 -4.763103008270264 -3.298583984375 -4.785400390625 -4.785888671875
8.61029052734375 -2.335145950317383 -4.481914043426514 -1.393310546875 -1.676025390625 -1.676025390625
8.595409393310547 -2.4251766204833984 -4.75587272644043 -2.468505859375 -4.822509765625 -4.822509765625
8.563671112060547 -2.4212870597839355 -4.871037483215332 -1.984619140625 -5.828765869140625 -5.829376220703125
8.547730445861816 -2.450873374938965 -4.763594627380371 -1.1191

8.620344161987305 -2.814356803894043 -5.54463005065918 -2.73193359375 -3.62060546875 -3.62109375
8.562841415405273 -2.8971810340881348 -5.703178882598877 -3.105712890625 -6.6524658203125 -6.653076171875
8.627958297729492 -2.775369644165039 -5.5338311195373535 -1.959228515625 -3.015869140625 -3.016357421875
8.54456901550293 -3.0070438385009766 -5.876341819763184 -3.815185546875 -6.230224609375 -6.229736328125
8.562202453613281 -2.9389114379882812 -5.716494560241699 -4.7254638671875 -4.9794921875 -4.980224609375
8.534667015075684 -3.053828716278076 -5.8160600662231445 -0.837646484375 -2.489990234375 -2.489990234375
8.545547485351562 -3.0038723945617676 -5.818419456481934 -2.08740234375 -5.3536376953125 -5.3543701171875
8.516075134277344 -3.047595977783203 -5.999828815460205 -2.902099609375 -4.85552978515625 -4.85504150390625
8.536210060119629 -3.2245750427246094 -6.295122146606445 -3.2421875 -5.1195068359375 -5.1195068359375
8.540884017944336 -3.143442153930664 -6.177886486053467 -3.5307

8.580732345581055 -3.246568202972412 -6.282555103302002 -4.5888671875 -3.03759765625 -3.037353515625
8.562610626220703 -3.118316650390625 -6.209273338317871 -1.965576171875 -13.982666015625 -13.983154296875
8.56717300415039 -3.1666274070739746 -6.2608842849731445 -2.76904296875 -4.889892578125 -4.888916015625
8.568674087524414 -3.380458116531372 -6.659754753112793 -3.2353515625 -6.73291015625 -6.732177734375
8.599153518676758 -3.333975315093994 -6.369943618774414 -2.3017578125 -12.5185546875 -12.518310546875
8.572102546691895 -3.2196438312530518 -6.313735485076904 -1.5869140625 -5.496826171875 -5.495849609375
8.591615676879883 -3.350872039794922 -6.4947590827941895 -2.3297119140625 -3.6732177734375 -3.6724853515625
8.532533645629883 -3.05850887298584 -6.052361488342285 -2.69970703125 -6.270751953125 -6.270263671875
8.57042121887207 -3.1913344860076904 -6.228743553161621 -3.79345703125 -3.182373046875 -3.1826171875
8.580364227294922 -3.047238826751709 -6.096273899078369 -1.578125 -2.970

In [3]:
torch.save('TSP_100points_small_8.5')

<All keys matched successfully>

34.396053314208984 -756.67529296875 -1242.8995361328125 -1242.8995361328125 -1242.8995361328125 -1242.8963623046875
34.175567626953125 -724.416015625 -1198.515625 -1198.515625 -1198.515625 -1198.5166015625
33.99101257324219 -768.504150390625 -1298.3597412109375 -1298.3597412109375 -1298.3597412109375 -1298.361083984375
35.05009460449219 -761.706298828125 -1300.712158203125 -1300.712158203125 -1300.712158203125 -1300.710693359375
34.037841796875 -755.2255859375 -1277.2818603515625 -1277.2818603515625 -1277.2818603515625 -1277.2823486328125
33.95484161376953 -749.0321655273438 -1302.9931640625 -1302.9931640625 -1302.9931640625 -1302.99169921875
35.48434829711914 -778.0559692382812 -1275.3516845703125 -1275.3516845703125 -1275.3516845703125 -1275.3514404296875
35.123233795166016 -754.6593017578125 -1263.115966796875 -1263.115966796875 -1263.115966796875 -1263.114990234375
34.516578674316406 -769.0162963867188 -1285.5791015625 -1285.5791015625 -1285.5791015625 -1285.57958984375
33.92599487

520.3328247070312 8.814233779907227
520.2237548828125 9.119690895080566
521.27294921875 9.099557876586914
521.3233032226562 8.788272857666016
520.3955078125 8.140579223632812
521.5120239257812 8.810598373413086
520.7025146484375 8.169943809509277
521.4142456054688 8.6790189743042
520.1701049804688 8.334620475769043
520.981689453125 8.485818862915039


In [ ]:
def evaluate(epochs = 30000, npoints = 10, batchsize = 100, nsamples = 8, negative_cutoff = 1):
    NN.eval()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(0, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        env.laststep()
        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())


def random_insertion(npoints=100, batchsize=200):
    env.reset(npoints, batchsize, nsamples=1)
    for i in range(npoints):
        env.update(torch.full([batchsize], i, dtype=torch.long, device=device))
    print(env.cost.mean().item(), env.cost.var().sqrt().item())